In [4]:
import time
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
from pandas import Series
from urllib import urlopen

In [5]:
##File with Teams/URLs to fetch
##CSV with a two columns(Link,Team_Name)
##Link is the location where the coach data is located
##Team is the name of the team the script is looking up
nfldata = pd.read_csv('nflstats.csv')

In [6]:
#Grabs the HTML table for each Link and appends them together
def footballquery(data):
    for i in range(1,32):
        html = urlopen(data.Link[i])
        soup = BeautifulSoup(''.join(html))
        table = soup.find('table',{'class':'sortable stats_table','id':'executives'})
        all_columns = []
        for row in table.findAll('tr'):
            col = row.findAll('td')
            columns = [c.getText().strip() for c in col]
            if(columns):
                all_columns.append(columns)
        df = pd.DataFrame(all_columns)
        df['Team'] = data.Team_Name[i]
        if i==1:
            full = df
        else:
            full = full.append(df)
    return full

In [7]:
##Grab the results and clear the index for the df
results = footballquery(nfldata)
results = results.reset_index(drop=True)

In [8]:
##Need to add rows for each of the coordinators, since they are currently on the same line as the heach coach
s = results[16].str.split('C:').apply(Series, 0).stack()
s.index = s.index.droplevel(-1)
s.name = 'CoachType'
s
final = results.join(s)
final = final.reset_index(drop=True)

In [9]:
###Now we just need to clean up coordinators with their appropriate titles
N = len(final)
for i in range(0,N):
    if final.CoachType[i] == "O" or final.CoachType[i] == "":
        final.CoachType[i] = "Head"
    if final.CoachType[i] == 'D':
        final.CoachType[i] = "Head"
    if final.CoachType[i].endswith('D'):
            temp = final.CoachType[i][:-1].lstrip()
            final[1][i] = temp
            final.CoachType[i]= "Offensive"
    else:
            if final.CoachType[i] != "Head":
                if 'OC' in final[16][i] and 'DC' in final[16][i]:
                        temp = final.CoachType[i].lstrip()
                        final[1][i] = temp
                        final.CoachType[i]= "Defensive"
                else:
                    if final[16][i].startswith('OC'):
                        temp = final.CoachType[i].lstrip()
                        final[1][i] = temp
                        final.CoachType[i]= "Offensive"
                    else:
                        if final[16][i].startswith('DC'):
                            temp = final.CoachType[i].lstrip()
                            final[1][i] = temp
                            final.CoachType[i]= "Defensive"

In [10]:
final.columns = ['year','coach','G','W','L','T','W-L%','G>.500','YR plyf','G plyf','W plyf','L plyf','W-L%','Chmp','SBwl','Conf','Coodinators','Team','CoachTitle']

In [11]:
#Save the table to a csv
final.to_csv('final.csv')